## **Installing Essential Packages**

In [ ]:
!pip install langchain-google-genai google-generativeai langchain langchain-community

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install google-search-results

In [ ]:
!pip install langchain_classic langchain-community

## **Importing LangChain/LangGraph Tools**

In [ ]:
# langgraph_rag_gemini.py
import os
from pathlib import Path
from typing import List, Any
from pydantic import BaseModel, Field

# LangGraph
from langgraph.graph import StateGraph, END

# LLM (Gemini)
from langchain_google_genai import ChatGoogleGenerativeAI

# Text splitting / embeddings / FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Retrieval chain
from langchain_classic.chains import RetrievalQA

In [ ]:
from google.colab import userdata
key=userdata.get('GAPI')

In [ ]:
# -------------------------
# Simple document loader + FAISS builder
# -------------------------
def load_txts(folder: str) -> List[str]:
    p = Path(folder)
    texts = []
    if not p.exists():
        return texts
    for f in p.glob("**/*.txt"):
        try:
            texts.append(f.read_text(encoding="utf-8"))
        except Exception:
            pass
    return texts

def build_faiss_from_texts(texts: List[str], persist_dir: str = "faiss_index"):
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
    chunks = []
    for t in texts:
        chunks.extend(splitter.split_text(t))

    embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.from_texts(chunks, embed)
    try:
        db.save_local(persist_dir)
    except Exception:
        pass
    return db


In [ ]:
# -------------------------
# Pydantic State model for LangGraph
# -------------------------
class RAGState(BaseModel):
    query: str = Field(...)
    action: str | None = None
    docs: List[Any] | None = None     # store Document-like objects
    answer: str | None = None

# -------------------------
# Setup LLM and FAISS (require GOOGLE_API_KEY)
# -------------------------

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, google_api_key=key)

# Build FAISS index from ./docs (or seed demo)
texts = load_txts("/content/data_store")

faiss_db = build_faiss_from_texts(texts)

# RetrievalQA chain (Gemini used inside RetrievalQA for synthesis)
retriever = faiss_db.as_retriever(search_kwargs={"k": 4})
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever)

# -------------------------
# Graph node functions
# -------------------------
def planner(state: RAGState) -> RAGState:
    q = state.query.lower()
    # simple heuristic planner
    if any(tok in q for tok in ("year", "released", "release", "when was")):
        state.action = "retrieve"
    else:
        state.action = "answer_direct"
    return state

def retriever_node(state: RAGState) -> RAGState:
    # use FAISS similarity search to get docs (documents have .page_content)
    docs = faiss_db.similarity_search(state.query, k=4)
    state.docs = docs
    return state

def answer_node(state: RAGState) -> RAGState:
    # build a short context and ask the LLM to answer grounded on it
    docs = state.docs or []
    context = "\n\n".join(getattr(d, "page_content", str(d)) for d in docs)
    prompt = f"Context:\n{context}\n\nQuestion:\n{state.query}\n\nAnswer concisely:"
    # call the qa_chain if available (it will use retriever internally) OR call llm with context
    # Prefer qa_chain.invoke to get the RetrievalQA behavior if supported
    if hasattr(qa_chain, "invoke"):
        raw = qa_chain.invoke({"query": state.query})
        # qa_chain.invoke may return dict or string; normalize in one line:
        text = str(getattr(raw, "content", raw))
    else:
        raw = llm.invoke(prompt)
        text = str(getattr(raw, "content", raw))
    state.answer = text
    return state


In [ ]:
# -------------------------
# Build LangGraph
# -------------------------
graph = StateGraph(RAGState)
graph.add_node("planner", planner)
graph.add_node("retriever", retriever_node)
graph.add_node("answer", answer_node)

graph.set_entry_point("planner")
# conditional edges from planner based on planner.action
graph.add_conditional_edges("planner", lambda s: s.action, {"retrieve": "retriever", "answer_direct": "answer"})
graph.add_edge("retriever", "answer")
graph.add_edge("answer", END)

app = graph.compile()

# -------------------------
# Run example
# -------------------------
import ast
if __name__ == "__main__":

    # interactive test
    while True:
        q = input("\nAsk (or 'exit'): ").strip()
        if q.lower() in ("exit", "quit"):
            break
        out = app.invoke({"query": q})
        print("\n=>", ast.literal_eval(out['answer'])['result'])
